In [1]:
import os
from openai import OpenAI
import json


api_key = "sk-proj-"

client = OpenAI(api_key=api_key)

In [2]:
from typing import List
from pydantic import BaseModel, Field

#### pydantic class

In [3]:
class pUser(BaseModel):
    name: str
    age: int
    email: str

In [4]:
foo_p = pUser(name="Jane", age=32, email="jane@gmail.com")

In [5]:
foo_p = pUser(name="Jane", age="bar", email="jane@gmail.com")

ValidationError: 1 validation error for pUser
age
  value is not a valid integer (type=type_error.integer)

In [6]:
class ListClass(BaseModel):
    students: List[pUser]

obj = ListClass(
    students=[pUser(name="Jane", age=32, email="jane@gmail.com")]
)

In [7]:
obj

ListClass(students=[pUser(name='Jane', age=32, email='jane@gmail.com')])

## Pydantic to OpenAI function definition

In [3]:
from langchain_core.utils.function_calling import convert_to_openai_function

In [4]:
# Define the function schema
class NewsArticle(BaseModel):
    """text to extract the author's name from"""
    author_name: str 


In [5]:
convert_to_openai_function(NewsArticle)

{'name': 'NewsArticle',
 'description': "text to extract the author's name from",
 'parameters': {'type': 'object',
  'properties': {'author_name': {'type': 'string'}},
  'required': ['author_name']}}

In [6]:
extract_author_name = convert_to_openai_function(NewsArticle)

In [7]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(api_key=api_key)

In [8]:
text = """
Hayati Akçay bu konuda şunları söyledi
Ordu'nun Fatsa ilçesinde şiddetli yağış su baskınlarına neden oldu.
20.04.2003
"""

In [9]:
res = model.invoke(text, functions=[extract_author_name])
res

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"author_name":"Hayati Akçay"}', 'name': 'NewsArticle'}}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 99, 'total_tokens': 118}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-6f61f206-3df0-423b-ac30-af49a7734f0c-0', usage_metadata={'input_tokens': 99, 'output_tokens': 19, 'total_tokens': 118})

In [10]:
res.additional_kwargs['function_call']

{'arguments': '{"author_name":"Hayati Akçay"}', 'name': 'NewsArticle'}

In [33]:
text = "hi"
res2 = model.invoke(text, functions=[extract_author_name])
res2

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 49, 'total_tokens': 59}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d02e2925-c277-4913-8a44-daa3f2ce3c42-0', usage_metadata={'input_tokens': 49, 'output_tokens': 10, 'total_tokens': 59})

### Forcing it to use a function

In [34]:
model_with_forced_function = model.bind(functions=[extract_author_name],
                                         function_call={"name":"NewsArticle"})

In [36]:
model_with_forced_function.invoke(text)

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"author_name":"Hayati Akçay"}', 'name': 'NewsArticle'}}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 101, 'total_tokens': 111}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-30360c55-487a-4a78-ba75-9fd8955eee57-0', usage_metadata={'input_tokens': 101, 'output_tokens': 10, 'total_tokens': 111})

###### random output

In [37]:
model_with_forced_function.invoke("hi!")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"author_name":"John Doe"}', 'name': 'NewsArticle'}}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 59, 'total_tokens': 66}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f98d0f62-18fe-4a21-8b1d-04148f6674ad-0', usage_metadata={'input_tokens': 59, 'output_tokens': 7, 'total_tokens': 66})

### Using chain

In [12]:
text1 = """
Hayati Akçay bu konuda şunları söyledi
Ordu'nun Fatsa ilçesinde şiddetli yağış su baskınlarına neden oldu.
20.04.2003
"""
text2 = """
Senem Yıldız bu konuyo doğruladı 
09.01.20024
"""

In [13]:
# Define the function schema
class NewsDate(BaseModel):
    """text to extract the author's name from"""
    article_date: str 

get_article_date = convert_to_openai_function(NewsDate)

res = model.invoke(text1, functions=[get_article_date])
res

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"article_date":"20.04.2003"}', 'name': 'NewsDate'}}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 99, 'total_tokens': 119}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-4874bed3-2e53-42a2-a519-f06e471a1e24-0', usage_metadata={'input_tokens': 99, 'output_tokens': 20, 'total_tokens': 119})

##### with retriver

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_texts(
    [text1, text2],
    embedding=OpenAIEmbeddings(api_key=api_key)
)
retriever = vectorstore.as_retriever()

In [48]:
retriever.get_relevant_documents("Hayati ne yazdı?")

/home/dell/miniconda3/envs/aa/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(page_content="\nHayati Akçay bu konuda şunları söyledi\nOrdu'nun Fatsa ilçesinde şiddetli yağış su baskınlarına neden oldu.\n20.04.2003\n"),
 Document(page_content='\nSenem Yıldız bu konuyo doğruladı \n09.01.20024\n')]

In [50]:
model_with_function = model.bind(functions=[get_article_date])

In [49]:
from langchain.prompts import ChatPromptTemplate


template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [51]:
from langchain.schema.runnable import RunnableMap

chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model_with_function 

In [54]:
res1 = chain.invoke({"question": "Hayati ne yazdı"})
res1

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"article_date":"20.04.2003"}', 'name': 'NewsDate'}}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 155, 'total_tokens': 175}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-0161e7d9-1cdd-4d1d-bb6f-5ad892fae095-0', usage_metadata={'input_tokens': 155, 'output_tokens': 20, 'total_tokens': 175})

In [55]:
res1.additional_kwargs["function_call"]

{'arguments': '{"article_date":"20.04.2003"}', 'name': 'NewsDate'}

### Multiple functions

In [14]:
functions = [
    convert_to_openai_function(NewsDate),
    convert_to_openai_function(NewsArticle),
]

In [24]:
multi_fun_model = model.bind(functions=functions)

In [25]:
res3 = multi_fun_model.invoke(text1)
res3

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"author_name":"Hayati Akçay"}', 'name': 'NewsArticle'}}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 124, 'total_tokens': 143}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-3c39bd8c-c9f3-4ef0-93de-e8553f704845-0', usage_metadata={'input_tokens': 124, 'output_tokens': 19, 'total_tokens': 143})

In [26]:
res3.additional_kwargs

{'function_call': {'arguments': '{"author_name":"Hayati Akçay"}',
  'name': 'NewsArticle'}}

#### forcing multiple func to use

In [27]:
multi_with_forced_function = model.bind(functions=functions,
                                         function_call={"name":"NewsArticle","name":"NewsDate"})

In [28]:
res4 = multi_with_forced_function.invoke(text1)
res4

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"article_date":"20.04.2003"}', 'name': 'NewsDate'}}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 133, 'total_tokens': 144}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-73e734de-1958-41fa-82af-e5cdb6acf95a-0', usage_metadata={'input_tokens': 133, 'output_tokens': 11, 'total_tokens': 144})

In [29]:
res4.additional_kwargs["function_call"]

{'arguments': '{"article_date":"20.04.2003"}', 'name': 'NewsDate'}